<a href="https://colab.research.google.com/github/slowbrain-jaym/Spheronisation-Inference/blob/main/Spheronisation_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spheronisation Inference Workbook

Workbook sets up a tensorflow object detection environment, loads a trained neural network and performs inference to locate and classify pellets within the spheroniser.

How to use:
1. Run the first 5 code cells up to the next text cell titled "inference". These code cells load the required libraries and files for inference.
2. These cells will also create a folder labelled "images" that you can access in the folder explorer on the left hand side of the colab page. Upload images for inference into this folder.
3. Run the inference cells below to create labelled versions of the images as well as a .csv file with all the detected pellets and their categories

First we create a folder tree and download the trained model and tensorflow object detection libraries

In [1]:
%mkdir workspace
%cd /content/workspace
%mkdir images results 

/content/workspace


In [2]:
!git clone --q https://github.com/tensorflow/models.git
!git clone --q https://github.com/slowbrain-jaym/SpheronisationCategorisation.git

Next we install the tensorflow dependencies

# New section

In [3]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -qq Cython contextlib2 pillow lxml matplotlib!pip install -qq pycocotools
%cd models/research/
!protoc object_detection/protos/*.proto --python_out=.

Selecting previously unselected package python-bs4.
(Reading database ... 144793 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpack .../5-python-html5lib_0.999999999-1_all.deb ...
Unpacking pyt

Installing tensorflow object detection and running the test function (this may take a couple of minutes to run)

If the installation is successful a message similar to "Ran 20 tests in 43.245s" will be displayed.

In [4]:
%cp object_detection/packages/tf2/setup.py .
!python -m pip install .
!python object_detection/builders/model_builder_tf2_test.py

Processing /content/workspace/models/research
     |████████████████████████████████| 8.7MB 7.4MB/s 
     |████████████████████████████████| 358kB 41.1MB/s 
     |████████████████████████████████| 849kB 36.2MB/s 
     |████████████████████████████████| 829kB 40.8MB/s 
     |████████████████████████████████| 61kB 5.9MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 61kB 4.7MB/s 
     |████████████████████████████████| 153kB 44.4MB/s 
     |████████████████████████████████| 63.8MB 60kB/s 
     |████████████████████████████████| 2.0MB 47.6MB/s 
     |████████████████████████████████| 174kB 40.8MB/s 
     |████████████████████████████████| 1.1MB 43.3MB/s 
     |████████████████████████████████| 36.7MB 88kB/s 
     |████████████████████████████████| 102kB 12.2MB/s 
     |████████████████████████████████| 112kB 46.4MB/s 
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1598976 sha256=bafceca732bed5132

In [6]:
%cd /content/workspace

/content/workspace


# Inference

Environment is now setup, we can perform inference.

Upload the images that inference is to be performed on to the "workspace/images" directory. Make sure that there are no non image files in this directory and there are no other directories within it.

# Outputs
Two files are output to the workspace/results folder:
* labelled_images.png - all the images for which inference has been performed with label boxes drawn on them, useful for visually inspecting how accurate the inference has been
* detections.csv - a list of all the objects detected with classification, certainty, coordinates and image name

In [7]:
# importing the required libraries

import os
os.environ['PYTHONPATH'] += ':/content/window_detection/models/:/content/window_detection/models/research/:/content/window_detection/models/research/slim/'
import pathlib
import tensorflow as tf
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL = "/content/workspace/SpheronisationCategorisation/mobilenet640x640/saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL) # loading the trained model

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...Done! Took 11.665812015533447 seconds


In [8]:
# loading the tensorflow label map file
PATH_TO_LABELS = '/content/workspace/SpheronisationCategorisation/mobilenet640x640/label_map.pbtxt'

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

In [11]:
image_path = "/content/workspace/images"
IMAGE_PATHS = [f for f in os.listdir(image_path) if os.path.isfile(os.path.join(image_path, f))] 
        # getting the file names for all the images in the image folder

In [12]:
# Perform inference on all the images in the images folder

import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

allresults = []
labelled_images = {}

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))


for image_name in IMAGE_PATHS:

    print('Running inference for {}... '.format(image_name), end='')

    image_path = "/content/workspace/images/" + image_name

    image_np = load_image_into_numpy_array(image_path)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)
    
    img_results = {}
    img_results["category"] = [category_index[id]['name'] for id in detections['detection_classes']]
    img_results["x1"] = [coord[0] for coord in detections['detection_boxes']]
    img_results["y1"] = [coord[1] for coord in detections['detection_boxes']]
    img_results["x2"] = [coord[2] for coord in detections['detection_boxes']]
    img_results["y2"] = [coord[3] for coord in detections['detection_boxes']]
    img_results["certainty"] = detections['detection_scores']
    img_results = pd.DataFrame(img_results)
    img_results["image"] = image_name

    allresults.append(img_results)

    plt.imshow(image_np_with_detections)
    plt.savefig("results/labelled_"+image_name, dpi=2000)
    print('Done')
#plt.show()

allresults = pd.concat(allresults)
allresults.to_csv("results/detections.csv")

Running inference for b (65).jpg... Done
Running inference for b (45).jpg... Done
Running inference for b (23).jpg... Done
Running inference for b (78).jpg... Done
Running inference for b (38).jpg... Done
Running inference for b (32).jpg... Done
Running inference for b (55).jpg... Done
Running inference for b (2).jpg... Done
Running inference for b (76).jpg... Done
Running inference for b (125).jpg... Done
Running inference for b (77).jpg... Done
Running inference for b (56).jpg... Done
Running inference for b.jpg... Done
Running inference for 5a 132dot8 crop.jpg... Done
Running inference for b (72).jpg... Done
Running inference for b (69).jpg... Done
Running inference for b (88).jpg... Done
Running inference for b (44).jpg... Done
Running inference for b (75).jpg... Done
Running inference for b (74).jpg... Done
Running inference for b (37).jpg... Done
Running inference for b (33).jpg... Done
Running inference for b (108).jpg... Done
Running inference for b (124).jpg... Done
Running in

Now results are generated you can zip them and download them, this may take a while

In [14]:
from google.colab import files
!zip -r /content/workspace/results.zip /content/workspace/results
files.download("/content/workspace/results.zip")

updating: content/workspace/results/ (stored 0%)
updating: content/workspace/results/labelled_b (87).jpg (deflated 9%)
updating: content/workspace/results/labelled_b (26).jpg (deflated 16%)
updating: content/workspace/results/labelled_b (46).jpg (deflated 13%)
updating: content/workspace/results/labelled_b (95).jpg (deflated 9%)
updating: content/workspace/results/labelled_b (45).jpg (deflated 11%)
updating: content/workspace/results/labelled_b (35).jpg (deflated 18%)
updating: content/workspace/results/labelled_b (127).jpg (deflated 13%)
updating: content/workspace/results/labelled_b (53).jpg (deflated 12%)
updating: content/workspace/results/labelled_b (55).jpg (deflated 13%)
updating: content/workspace/results/labelled_b (50).jpg (deflated 12%)
updating: content/workspace/results/labelled_b (90).jpg (deflated 10%)
updating: content/workspace/results/labelled_b (84).jpg (deflated 9%)
updating: content/workspace/results/labelled_b (38).jpg (deflated 17%)
updating: content/workspace/re

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>